# Setup

In [ ]:
import json
from IPython.display import display
from pydantic import BaseModel, Field
from typing import Any
import supabase
from supabase.lib.client_options import SyncClientOptions

import matplotlib.pyplot as plt

SUPABASE_URL_BASE = "https://znsozdvrmfdwxyymtgdz.supabase.co/"

with open("secrets.json", "r") as f:
    secrets = json.load(f)


sb_client = supabase.create_client(
    SUPABASE_URL_BASE,
    secrets["EEVA_SUPABASE_SERVICE_KEY"],
    options=SyncClientOptions(auto_refresh_token=False, persist_session=False),
)


class Profile(BaseModel):
    identity: float = Field(ge=0, le=1)
    horoscope: str = Field()

    def cmp(self, other: "Profile") -> float:
        return abs(self.identity - other.identity)

# Tools

### View questions

In [ ]:
questions = sb_client.table("questions").select("*").execute().data
questions

### Update questions

In [ ]:
if False:
    with open("data/questions.json", "r") as f:
        questions = json.load(f)
    sb_client.table("questions").upsert(questions, on_conflict="id").execute()

### Mark profiles as dirty

In [ ]:
if False:
    sb_client.functions.invoke("mark-profiles-dirty", {"body": {}})

### Get user data

In [ ]:
raw_answers = (
    sb_client.table("user_answers")
    .select("user_id, question_id, answer_text")
    .execute()
    .data
)
user_answer_lists: dict[str, dict[str, str]] = {}
for ans in raw_answers:
    user_answer_lists.setdefault(ans["user_id"], {})[ans["question_id"]] = ans[
        "answer_text"
    ]

raw_user_data = (
    sb_client.table("profiles")
    .select("user_id,first_name,last_name,hidden,profile")
    .execute()
    .data
)
users: dict[str, dict[str, Any]] = {}
for user in raw_user_data:
    user_id = user["user_id"]
    if user["hidden"] or user_id not in user_answer_lists:
        continue
    users[user_id] = {
        "name": f"{user['first_name']} {user['last_name']}",
        "profile": Profile.model_validate(user["profile"]),
        "answers": user_answer_lists[user_id],
    }

users

### Plot identity scores

In [ ]:
identity_scores = {user["name"]: user["profile"].identity for user in users.values()}

# Sort identity_scores by value
sorted_items = sorted(identity_scores.items(), key=lambda x: x[1])
sorted_names = [item[0] for item in sorted_items]
sorted_scores = [item[1] for item in sorted_items]

plt.figure(figsize=(12, 6))
plt.bar(sorted_names, sorted_scores)
plt.xlabel("User")
plt.ylabel("Identity Score")
plt.title("Identity Scores by User (Ordered)")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()